<a href="https://colab.research.google.com/github/rwalia2405/Decision_Trees/blob/master/Decision_tree_classifier_Using_SVM(2_Classes).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.w=[0]
        self.i=0
        self.left = None
        self.right = None

In [0]:
from sklearn import svm
model=svm.SVC(kernel='linear')
class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth and np.unique(y).shape[0]>1:
            model.fit(X,y)
            w=model.coef_
            i=model.intercept_
            desc=X.dot(np.transpose(w))+i
            indices_left = desc < 0
            # print("indices_left",indices_left.shape)
            node.w=w
            node.i=i
            indices_left=np.squeeze(indices_left)
            X_left, y_left = X[indices_left], y[indices_left]
            X_right, y_right = X[~indices_left], y[~indices_left]
            node.left = self._grow_tree(X_left, y_left, depth + 1)
            node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if (inputs.reshape(1,inputs.shape[0])).dot(np.transpose(node.w))+node.i < 0:
                node = node.left
            else:
                node = node.right
        return node.predicted_class


In [4]:
if __name__ == "__main__":
    import sys
    from sklearn import datasets
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    dataset = datasets.load_breast_cancer()
    X, y = dataset.data, dataset.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

    clf = DecisionTreeClassifier(max_depth=2)
    clf.fit(X_train, y_train)

    print("Accuracy:",accuracy_score(y_test,clf.predict(X_test)))

    # print(clf.predict(X_test))

Accuracy: 0.951048951048951
